# comercio_ext_indices.tb_referencia_nbm_ncm
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/NBM_NCM_delta`
## 📌 Descrição do arquivo

Tabela de referência (bridge) que mapeia **códigos NBM** (legado) para **códigos NCM** (atual/padrão do COMEX moderno).

Tipo: Bridge / Crosswalk de produto (NBM → NCM)

|Coluna|Descrição técnica|Interpretação humana|
|---|---|---|
|`CO_NBM`|Código do produto na NBM|Código legada do item|
|`CO_NCM`|Código do produto na NCM|Código padronizado atual equivalente|

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:
import org . apache . spark  . sql . functions . _
import org . apache . spark  . sql . types . _
import org . apache . spark  . sql . DataFrame
import org . apache . hadoop . fs  . FileSystem 
import org . apache . hadoop . fs  . Path 
import io  . delta  . tables . _

In [0]:

val bronzePath =     "/mnt/bronze/autoloader/landingbeca2026jan/balancacomercial/NBM_NCM_delta/"   
val silverPath =     "/mnt/silver/landingbeca2026jan/comercio_ext_indices/tb_referencia_nbm_ncm/"   
val silverTable =    "tb_referencia_nbm_ncm"
val fs = FileSystem.get(spark.sparkContext.hadoopConfiguration)



In [0]:

val silverSchema = StructType(
    Seq (
      StructField ( "CO_NBM" , StringType ,  nullable = false ) ,
      StructField ( "CO_NCM" , StringType ,  nullable = false ) ,
      StructField ( "TS_REF" , TimestampType ,  nullable = false ) ,
      StructField ( "NM_ORIGEM" , StringType ,  nullable = false ) 
    )
  )

## Extração
> #### **saprk.read**

In [0]:
val dfBronzeRaw = spark . read . format ( "delta" ) . load ( bronzePath )

##Normalização
> #### **datatype**
> #### **regras**

In [0]:
val dfNormalized = dfBronzeRaw
  .withColumn("CO_NBM", trim(upper(col("CO_NBM").cast(StringType))))
  .withColumn("CO_NCM", trim(upper(col("CO_NCM").cast(StringType))))

In [0]:
val dfWithDefaults = dfNormalized
  .withColumn("TS_REF", current_timestamp())
  .withColumn("NM_ORIGEM", lit("/landingbeca2026jan/balancacomercial/NBM_NCM_delta"))

##Validações
> #### **data quality**
> #### **deduplicação**
> #### **schema fit**

In [0]:

val dfValid = dfWithDefaults . filter ( 
    col("CO_NBM").isNotNull && col("CO_NCM").isNotNull
  )


In [0]:
val dfDedup = dfValid . dropDuplicates (
    Seq (
        "CO_NBM",
        "CO_NCM"
    )
  )

In [0]:
val dfSilver = dfDedup.select( col("CO_NBM"), col("CO_NCM"), col("TS_REF"), col("NM_ORIGEM") )

##Carga
> #### **overwrite**

In [0]:
spark.conf.set("spark.databricks.optimizer.dynamicPartitionPruning", "true")
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

val silverExists = fs.exists(new Path(silverPath))

if (!silverExists) {
  dfSilver
    .write
    .format("delta")
    .mode("overwrite")
    .save(silverPath)
}